In [ ]:
!pip install -q gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.7 MB/s eta 0:

In [ ]:
import torch
import torchtext
from torchtext import data
import torch.optim as optim
import argparse
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

# 3.3.1
# The first time you run this will download a 862MB size file to .vector_cache/glove.6B.zip
glove = torchtext.vocab.GloVe(name="6B",dim=100) # embedding size = 100

.vector_cache/glove.6B.zip: 862MB [02:40, 5.38MB/s]                           
100%|█████████▉| 399999/400000 [00:25<00:00, 15472.28it/s]


In [ ]:
class baselineModel(torch.nn.Module):
  def __init__(self, vocab, embedding_size):
    super().__init__()

    self.embedding = torch.nn.Embedding.from_pretrained(vocab.vectors)
    self.out = torch.nn.Linear(embedding_size, 1)


  def forward(self, x):
    e = self.embedding(x)
    average = torch.mean(e, 0, True)
    logits = self.out(average)
    return logits.reshape([-1])

In [ ]:
class CNNModel(torch.nn.Module):
  def __init__(self, vocab,k1,k2,n1,n2):
    super().__init__()
    self.k1 = (k1, 100)
    self.k2 = (k2, 100)
    self.n1 = n1
    self.n2 = n2
    self.probabilityFunction = torch.nn.Sigmoid()

    self.embedding = torch.nn.Embedding.from_pretrained(vocab.vectors)

    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=self.n1, kernel_size=self.k1, bias=False)
    self.bn1 = torch.nn.BatchNorm2d(self.n1)
    self.maxpool1 = torch.nn.AdaptiveMaxPool2d(output_size=(1, 1))

    self.conv2 = torch.nn.Conv2d(in_channels=1, out_channels=self.n2, kernel_size=self.k2, bias=False)
    self.bn2 = torch.nn.BatchNorm2d(self.n2)
    self.maxpool2 = torch.nn.AdaptiveMaxPool2d(output_size=(1, 1))

    self.out = torch.nn.Linear(self.n1+self.n2, 1)

  def forward(self, x):
    e = self.embedding(x)
    input = torch.transpose(e, 0, 1).unsqueeze(1)
    x1 = self.conv1(input)
    x1 = F.relu(x1)
    x1 = self.bn1(x1)
    x1 = self.maxpool1(x1)

    x2 = self.conv2(input)
    x2 = F.relu(x2)
    x2 = self.bn2(x2)
    x2 = self.maxpool2(x2)

    concatenate = torch.cat((x1, x2), dim=1)
    output = self.out(concatenate.squeeze())
    logits = self.probabilityFunction(output)

    return logits.reshape([-1])

In [ ]:
def runModel(sentence,type = 'Baseline Model'):
  probabilityFunction = torch.nn.Sigmoid()

  if type == 'Baseline Model':
    checkpoint = torch.load('baselineModel.pt')
    model = baselineModel(glove,glove.vectors.shape[1])
    model.load_state_dict(checkpoint)
    model.eval()

    tokens = sentence.split()
    token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
    token_tensor = torch.LongTensor(token_ints).view(-1,1)

    logit = model(x=token_tensor)
    probability = probabilityFunction(logit)
    probability = torch.maximum(probability, torch.tensor([1e-5]))
    probability = torch.minimum(probability, torch.tensor([0.99999]))
    Y_pred = torch.round(probability)

  elif type == 'CNN Model':
    checkpoint = torch.load('CNNModel.pt')
    model = CNNModel(glove,k1=2,k2=4,n1=20,n2=20)
    model.load_state_dict(checkpoint)
    model.eval()

    tokens = sentence.split()
    token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
    token_tensor = torch.LongTensor(token_ints).view(-1,1)

    mark = 0
    if len(token_tensor) < 4:
      temp = []
      for i in range(len(token_tensor)):
        temp.append([int(token_tensor[i].detach().numpy())])
      while(mark == 0):
        if len(temp) < 4:
          temp.append([1])
        else:
          mark = 1
      token_tensor = torch.tensor(temp)
    logit = model(x=token_tensor)
    probability = probabilityFunction(logit)
    probability = torch.maximum(probability, torch.tensor([1e-5]))
    probability = torch.minimum(probability, torch.tensor([0.99999]))
    Y_pred = torch.round(probability)

  if Y_pred == 1:
    result = 'Subjective'
  elif Y_pred == 0:
    result = 'Objective'

  return result

In [ ]:
runModel('moon','CNN Model')

'Objective'

In [ ]:
runModel('hatred','CNN Model')

'Subjective'

In [ ]:
# please upload baselineModel.pt and CNNModel.pt into colab
gr.Interface(fn=runModel,
      inputs=["text",
      gr.Radio(['Baseline Model', 'CNN Model'])
      ],
      outputs="text").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5f01d31881ccb39456.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
